In [1]:
import javalang
import os
import chardet
import mes_Depatch as dp
import python_debugger as pyd
import random
import testNormalizeCode as nor
import testAugmentedCode as aug

import xml.etree.ElementTree as ET
import numpy as np
import glob

In [2]:
def listdir_fullpath(d):
    return [f for f in os.listdir(d)]

In [3]:
def augmentCode(sourcefileX, changeVariable,permuStatement, task=1 ):
    data=pyd.readJavaFile(sourcefileX)
    tree = javalang.parse.parse(data)
    newCode,_, permutation = pyd.AST(tree).getAugmentedCode(task = task, changeVariable=changeVariable, permuStatement=permuStatement)
    return newCode,permutation

In [4]:
def changeLineBlock(begin, end, permutation):
    #print("permutation:",permutation)
    permu_lineblock = [value for value in range(begin, end+1)]#make permu_lineblock get permutation value    
    #print("permu_lineblock: ",permu_lineblock)
    for i in range(len(permu_lineblock)):
        #print("傳換前:",permu_lineblock[i])
        permu_lineblock[i] = permutation[permu_lineblock[i]] 
        #print("傳換前:",permu_lineblock[i])
    permu_lineblock.sort()#sort permu
    line = [] 
    lines = []
    last_permu = (len(permu_lineblock)-1)
    for i in range(len(permu_lineblock)):
        if (i == 0):
            line.append(permu_lineblock[i])
            if (permu_lineblock[i] == permu_lineblock[last_permu]):
                lines.append(list(line))
                line.clear()
        elif (permu_lineblock[i] == (permu_lineblock[i-1]+1) ):
            line.append(permu_lineblock[i])
            if (permu_lineblock[i] == permu_lineblock[last_permu]):
                lines.append(list(line))
                line.clear()
        elif (permu_lineblock[i] != (permu_lineblock[i-1]+1) ):
            lines.append(list(line))
            line.clear()
            line.append(permu_lineblock[i])
            if (permu_lineblock[i] == permu_lineblock[last_permu]):
                lines.append(list(line))
                line.clear()
        else:
            line.append(permu_lineblock[i])
            lines.append(line)
            line.clear()
    return lines

In [5]:
def inputGenerate(list_newCode,copyNum,txtfiledir,case):
    '''
    fname = os.path.join(txtfiledir,str(copyNum))
    copyfiledir = txtfiledir + "\\" + str(copyNum)  #create txt case/0 dir, e.g. XX/CASE/0
    #print("fname",fname)
    #print("copyfiledir",copyfiledir)
    if not os.path.exists(fname):
        os.mkdir(fname)'''
    fname = os.path.join(txtfiledir,case)
    f_input=open(fname+".txt",'w',encoding="utf-8") #寫檔
    for newCode in list_newCode:
        f_input.write(" <BOC> ")
        NEWLINE = 	"abcdefgijkoooopppp"
        for content in newCode:
            if(content==NEWLINE):
                f_input.write("\n")
                f_input.write(" <CR> ")
            else:
                f_input.write(content)
                f_input.write(" ")
            #print(content)
        f_input.write(" <EOC> ")
    f_input.close()

In [6]:
def parseSentence(x):	
    result = ""
    prev=""
    for i in range(len(x)):
            #print("x[i]: ", x[i])
            #print("prev: ", prev)
            if (ord(x[i])>255): #"U"
                #print("current : U")
                if (prev=="U"):
                    result += x[i]
                elif (prev=="A"):
                    if x[i-1]==" ":
                        result +=  x[i]
                    else:
                        result += " " + x[i]                        
                else:
                    result += x[i]
                prev ="U"
            else: #"A"
                #print("current : A")
                if (prev=="U"):
                    if (x[i]==" "):
                        result +=  x[i]
                    else:
                        result += " " + x[i]
                elif (prev=="A"):
                    result += x[i]
                else:
                    result += x[i]
                prev ="A"
            #print("result: ", result)

    return result

In [7]:
mapper={"正確:正確": 1,"變數:重複宣告變數": 2,"變數:變數未定義無法使用": 3,"變數:不了解局部變數的概念": 4,"運算:運算錯誤:型態不符無法運算":5,"函數:未傳遞參數或傳回值": 6,"函數:函數庫使用錯誤": 7,"函數:函數結構不清楚": 8,"函數:函數概念不清楚": 9,"字串:字串比較錯誤(沒有使用equals函數)": 10,"io:不會使用輸入scanner類別":11,"io:重複宣告scanner物件":12,"物件:物件動作概念(建構子)": 13,"物件:物件資料/物件動作概念": 14,"物件:正確的產生物件":15,"物件:無法分辨物件&類別的資料/動作":16,"物件:建立物件之間的關係":17,"物件:不會使用物件之間的關係":18,"物件:不會使用物件資料":19,"陣列:不會宣告陣列變數":20,"陣列:陣列索引值":21,"陣列:陣列溢位":22,"其他:撰寫main函式":23,"其他:關鍵字錯誤":24,"其他:筆誤":25,"其他:不熟悉JAVA程式結構":26,"繼承:使用繼承":27,"繼承:使用super產生繼承物件":28,"抽象:繼承資料或動作":29,"抽象:抽象動作":30,"抽象:抽象類別不能直接產生物件":31,"抽象:覆寫抽象動作":32,"抽象:不會分離變跟不變的部分":33,"介面:Comparable介面":34,"介面:Comparable介面的排序應用":35,"迴圈:迴圈概念錯誤":36}

In [8]:
print("mapper",mapper)

mapper {'正確:正確': 1, '變數:重複宣告變數': 2, '變數:變數未定義無法使用': 3, '變數:不了解局部變數的概念': 4, '運算:運算錯誤:型態不符無法運算': 5, '函數:未傳遞參數或傳回值': 6, '函數:函數庫使用錯誤': 7, '函數:函數結構不清楚': 8, '函數:函數概念不清楚': 9, '字串:字串比較錯誤(沒有使用equals函數)': 10, 'io:不會使用輸入scanner類別': 11, 'io:重複宣告scanner物件': 12, '物件:物件動作概念(建構子)': 13, '物件:物件資料/物件動作概念': 14, '物件:正確的產生物件': 15, '物件:無法分辨物件&類別的資料/動作': 16, '物件:建立物件之間的關係': 17, '物件:不會使用物件之間的關係': 18, '物件:不會使用物件資料': 19, '陣列:不會宣告陣列變數': 20, '陣列:陣列索引值': 21, '陣列:陣列溢位': 22, '其他:撰寫main函式': 23, '其他:關鍵字錯誤': 24, '其他:筆誤': 25, '其他:不熟悉JAVA程式結構': 26, '繼承:使用繼承': 27, '繼承:使用super產生繼承物件': 28, '抽象:繼承資料或動作': 29, '抽象:抽象動作': 30, '抽象:抽象類別不能直接產生物件': 31, '抽象:覆寫抽象動作': 32, '抽象:不會分離變跟不變的部分': 33, '介面:Comparable介面': 34, '介面:Comparable介面的排序應用': 35, '迴圈:迴圈概念錯誤': 36}


In [9]:
'''list_src_dict={}
mes = {'i': 'R6', 'args': 'P5', 'type': 'l4', 'name': 'Z', 'list1': 'Q4', 'id': 'X7', 'kind': 'u1', 'list2': 'a8', 'tel': 'l0', 'pid': 'w9'}
src_dict ={'input': mes}
list_src_dict.update(src_dict)
mes ={'id': 'S2', 'tel': 'l6'}
src_dict ={'keeper': mes}
list_src_dict.update(src_dict)
mes ={'name': 'F9', 'pid': 'm5', 'type': 'S4'}
src_dict ={'pet': mes}
list_src_dict.update(src_dict)
print("list_src_dict",list_src_dict)'''
list_src_dict={}
mes = {'i': 'R6', 'args': 'P5', 'type': 'l4', 'name': 'Z', 'list1': 'Q4', 'id': 'X7', 'kind': 'u1', 'list2': 'a8', 'tel': 'l0', 'pid': 'w9'}
list_src_dict.update(mes)
print("list_src_dict",list_src_dict)
mes2 ={'id': 'S2', 'tel': 'l6'}
list_src_dict.update(mes2)
print("list_src_dict",list_src_dict)
mes.update(mes2)
print("mes",mes)

list_src_dict {'i': 'R6', 'args': 'P5', 'type': 'l4', 'name': 'Z', 'list1': 'Q4', 'id': 'X7', 'kind': 'u1', 'list2': 'a8', 'tel': 'l0', 'pid': 'w9'}
list_src_dict {'i': 'R6', 'args': 'P5', 'type': 'l4', 'name': 'Z', 'list1': 'Q4', 'id': 'S2', 'kind': 'u1', 'list2': 'a8', 'tel': 'l6', 'pid': 'w9'}
mes {'i': 'R6', 'args': 'P5', 'type': 'l4', 'name': 'Z', 'list1': 'Q4', 'id': 'S2', 'kind': 'u1', 'list2': 'a8', 'tel': 'l6', 'pid': 'w9'}


In [24]:
f_output=open("732.txt",'w',encoding="utf-8")
tree = ET.parse("732.xml")
root = tree.getroot()
for Error in root.iter("Error"):
    f_output.write(" <BOTM> ")
    f_output.write(" <BOT> ")
    error_type=Error.get("tpye")
    #print("error_type",error_type)
    new_type=mapper[error_type]
    f_output.write(str(new_type))
    f_output.write(" <EOT> ")
    #print("new_type",new_type)
    message = Error.find('Message').text
    #print("message",message)
    #list_message = parseSentence(message)
    list_message = message.split('\n')
    '''for token in list_message:
        print(token,"___")'''
    f_output.write(" <BOM> ")
    for token in list_message:
        #print("type(token)",type(token))
        #print("\\\\")
        if token in mes:
            print("變化前token",token)
            token=mes[token]
            f_output.write(token)
            print("變化後token",token)
        else:
            #print(token)
            f_output.write(token)
    f_output.write(" <EOM> ")
    f_output.write(" <EOTM> ")
f_output.close()            
    
    
    

In [ ]:
'''ref https://officeguide.cc/python-read-write-xml-format-file-tutorial-examples/'''
def xmlGenerate(xml_name,xml_source_dir,list_src_dict,copyNum,output_copyfiledir,case):
    '''
    copyfiledir = os.path.join(xmlfiledir,str(copyNum))
    #copyfiledir = xmlfiledir + "\\" + str(copyNum)  #create txt case/0 dir, e.g. XX/CASE/0
    if not os.path.exists(copyfiledir):
        os.mkdir(copyfiledir)'''
    #print("in xmlGenerate ->filename:",xml_source_dir)
    tree = ET.parse(xml_source_dir)#抓到xml檔
    root = tree.getroot()#找到根節點
    for Error in root.iter("Error"):#root 尋找 Error 節點
        new_Linelist=ET.SubElement(Error,"new_Linelist")#在Error這個節點創建子節點new_Linelist提供給轉換過後的段落存取
        for Linelist in Error.iter("Linelist"):#Error 尋找 Linelist 節點
            for line in Linelist.iter("line"):#在Linelist 尋找 Line 節點
                #追蹤用   print("line.attrib",line.attrib) #印出line的屬性 
                old_begin=line.get("Begin")# 取得節點指定屬性質 此時抓到的type為str
                new_begin=int(old_begin)#創建一個新變數將old_begin轉換成int
                old_end=line.get("End")
                new_end=int(old_end)
                src=line.get("src")
                #追蹤用   print("src",src)
                if(new_begin == new_end == -1):#當new_begin&new_end都為-1時沒辦法丟入changeLineBlock()去轉換新的段落，所以直接拿原檔的行號做輸出
                    #print("new_begin: ",new_begin,", new_end: ",new_end)
                    element=ET.SubElement(new_Linelist,"new_line")#將產生出的新段落放入new_Linelist的子節點
                    element.set('Begin',old_begin)#因為tree.write()沒辦法參照"-1"的int值所以拿沒轉換過的old_begin的str值設值
                    element.set('End',old_end)#因為tree.write()沒辦法參照"-1"的int值所以拿沒轉換過的old_end的str值設值
                    element.set('src',src)
                else:
                    line_blocks=changeLineBlock(new_begin,new_end,list_src_dict[src])#進入changeLineBlock將舊段落對照permutation產生新的段落   
                    #追蹤用   print("line_blocks",line_blocks)
                    for line_block in line_blocks:
                        element=ET.SubElement(new_Linelist,"new_line")#將產生出的新段落放入new_Linelist的子節點
                        element.set('Begin',str(line_block[0]))#設置節點的屬性 因為line_block裡面是list ex:[2,3,4,5,6] 所以抓的是list初始位置
                        element.set('End',str(line_block[-1]))#設置節點的屬性 因為line_block裡面是list ex:[2,3,4,5,6] 所以抓的是list最後位置
                        element.set('src',src)
                        #追蹤用   print("element.attrib",element.attrib)


    for Error in root.iter("Error"):#將原本Linelist的每個line節點作清空
        for Linelist in Error.iter("Linelist"):
            for line in Linelist.iter("line"):
                line.clear()
    for Error in root.iter("Error"):#把Error裡的每個Linelist節點刪除
        for Linelist in Error.iter("Linelist"):
            Error.remove(Linelist)

    for Error in root.iter("Error"):#先將每個new_Linelist底下的new_line的data改名為line
        for new_Linelist in Error.iter("new_Linelist"):
            for new_line in new_Linelist.iter("new_line"):
                new_line.tag="line"
    for Error in root.iter("Error"):
        for new_Linelist in Error.iter("new_Linelist"):#再將每個Error底下的new_Linelist的data改名為Linelist
                new_Linelist.tag="Linelist"

    '''note
    創一個新的節點為"new_Linelist"(在Linelist之外)
    將轉換過後的new_line放到new_Linelist
    把line做clear->把Linelist做clear
    把new_line改名為line->把new_Linelist改名為Linelist
    '''
    fname=os.path.join(output_copyfiledir,xml_name)
    tree.write(fname)

In [ ]:
source_dir = "D:\\NewVerPythonDebugger\\SourceCODES"
NormalizedDir = "D:\\NewVerPythonDebugger\\NormalizedCODES"
txt_dir = "D:\\NewVerPythonDebugger\\InputTxt"
xml_dir = "D:\\NewVerPythonDebugger\\OutputXml"
PatchBackDir =  "D:\\NewVerPythonDebugger\\PatchBackCODES"
PatchedDir = "D:\\NewVerPythonDebugger\\PatchCODES"
cases = listdir_fullpath(source_dir)
copyNum = 1

#print("cases",cases)
for case in cases:
    print("case",case)
    javafiles=[]
    list_newCode=[]
    list_src_dict={}
    sourcefiledir = os.path.join(source_dir,case)
    pachedfiledir = os.path.join(PatchedDir,case)
    normalfiledir = os.path.join(NormalizedDir,case)
    if not os.path.exists(normalfiledir):
            os.mkdir(normalfiledir)
    #print("sourcefiledir",sourcefiledir)
    txtfiledir = os.path.join(txt_dir,case)
    #print("txtfiledir",txtfiledir)
    if not os.path.exists(txtfiledir):
            os.mkdir(txtfiledir)
    xmlfiledir = os.path.join(xml_dir,case)
    #print("xml_dir",xml_dir)
    #print("xmlfiledir",xmlfiledir)
    if not os.path.exists(xmlfiledir):
        os.mkdir(xmlfiledir)
    sourcefiles = listdir_fullpath(sourcefiledir)
    #print("sourcefiles",sourcefiles)
    '''
    for sourcefile in sourcefiles:
        if sourcefile.endswith('.java'):
            src_filename = sourcefiledir + "\\" + sourcefile
            patched_filename = pachedfiledir + "\\" + sourcefile
            normalized_filename = normalfiledir + "\\" + sourcefile
            print("src_filename: ", src_filename)
            #Step 1. compute normalized code
            newCode = nor.normalizeCode(patched_filename) #normalized code
            #myprint("normalized code: ", newCode)
            pyd.write_to_file(newCode, normalized_filename) #save to normalized code dir'''
            #transed_code =dp.transformJavaCode( patchRule, normalized_filename, True, True, task=2 ) #back to syntatic-error with normalized code
        
    for i in range(copyNum):
        file_count=0
        for sourcefile in sourcefiles:
            if sourcefile.endswith('.java'):
                src_filename = sourcefiledir + "\\" + sourcefile
                patched_filename = pachedfiledir + "\\" + sourcefile
                normalized_filename = normalfiledir + "\\" + sourcefile
                print("src_filename: ", src_filename)
                #Step 1. compute normalized code
                newCode = nor.normalizeCode(patched_filename) #normalized code
                #myprint("normalized code: ", newCode)
                pyd.write_to_file(newCode, normalized_filename) #save to normalized code dir
                #transed_code =dp.transformJavaCode( patchRule, normalized_filename, True, True, task=2 ) #back to syntatic-error with normalized code
                src=sourcefile.split(".java")
                print("src[0]",src[0])
                src_code = pyd.readJavaFile(src_filename)
                patchRule =  dp.getTransformRule(normalized_filename, src_filename)
                print("file_count",file_count)
                if(file_count==0):
                    transed_code, varSet=dp.transformJavaCode( patchRule, normalized_filename, True, True, task=2 ) #back to syntatic-error with normalized code
                else:
                    transed_code, varSet=dp.transformJavaCode( patchRule, normalized_filename, True, True, task=2 ,varSet=varSet)
                file_count=file_count+1
                list_newCode.append(transed_code)
                #print("list_newCode",list_newCode)
                print("varSet",varSet)
                '''src_dict={src[0]:varSet}
                print("src_dict",src_dict)
                list_src_dict.update(src_dict)'''
                #print("list_src_dict",list_src_dict)
            elif sourcefile.endswith('.xml'):
                xmlfile=sourcefile
            
        input_copyfiledir = os.path.join(txtfiledir,str(i))  #create dest case/0 dir, e.g. XX/CASE/0
        #print("input_copyfiledir",input_copyfiledir)
        if not os.path.exists(input_copyfiledir):
            os.mkdir(input_copyfiledir)
        output_copyfiledir = os.path.join(xmlfiledir,str(i))  #create dest case/0 dir, e.g. XX/CASE/0
        #print("output_copyfiledir",output_copyfiledir)
        if not os.path.exists(output_copyfiledir):
            os.mkdir(output_copyfiledir)
        '''
        for javafile in javafiles:
            #print("javafile",javafile)
            jfname = os.path.join(sourcefiledir,javafile)
            newCode, permutation = augmentCode(jfname, True, True, 1)
            #print("type(sourcefile)",type(sourcefile))
            src=javafile.split(".java")
            print("src[0]",src[0])
            list_newCode.append(newCode)
            #print("list_newCode",list_newCode)
            src_dict={src[0]:permutation}
            list_src_dict.update(src_dict)
            print("list_src_dict",list_src_dict)'''
        inputGenerate(list_newCode,copyNum,input_copyfiledir,case)
        xmlfile_dir = os.path.join(sourcefiledir,xmlfile)
        #print("xml_dir",xmlfile_dir)
        #xmlGenerate(xmlfile,xmlfile_dir,list_src_dict,copyNum,output_copyfiledir,case)
        
        #print("javafiles",javafiles)
        #print("xmlfile",xmlfile)